In [1]:
import json
import re
import findVorgangRegex as vRex
import os
from tika import parser

In [2]:
findVorgang= vRex.findVorgangRex

In [3]:
 #target_dir = r"C:\Users\schull\Projekte\KIbarDok\testDatenMerge"
target_dir = r"C:\Users\guendm1\Documents\BerlinDenkmal\Testdaten\All\AllTxt"
# list all documents
os.chdir(target_dir)
files = sorted(os.listdir(target_dir))

In [4]:
def filename_divider(filename):
    fname, fext = os.path.splitext(filename)
    #print('before: ',fname)
    fnameNew = fname.replace(".", " ").replace("_", " ").replace("-", " ").replace("'", " ").replace(","," ")
    #print('after: ',fnameNew)
    return fnameNew

In [5]:
#Hauptanalyse: Treffer-Analyse
m=0
n=0
wrongFormatList=[]
matchList={}
noMatchList={}
for i in range(0,len(files)):
    print('i: ',i,'n: ',n,'m: ',m,files[i])
    # print("Processing file:", files[i], i)
    
    file_extension = files[i][-3:].lower()
            
    raw = parser.from_file(files[i])
    ########################find amount of pages
    try:
        pages=raw['metadata']['xmpTPg:NPages']
        if type(pages)==str:
            pages=int(pages)
        else:
            pages=int(pages[-1])
    except:
        pages=1
    ############################################
    if pages<=15:
        inhalt = raw['content']
    else:
        inhalt=''
    ############################################
    if inhalt == None or inhalt =='':
            print("File does not contain any text data:", files[i])
            wrongFormatList.append(files[i])
            continue
    ##################################################remove unecessary spaces
    inhalt = inhalt + ' ' + filename_divider(files[i])
    inhalt = inhalt.replace("\n", " ")
    inhalt=' '.join(inhalt.split())
    ##################################################Begin - Analyse the file name
    if files[i].count(' ')<5:
        if 'Anfr' in files[i]:
            matchList[files[i]]={'Anfrage':['Dateiname']}
            continue
        elif 'Anhö' in files[i]:
            matchList[files[i]]={'Anhörung':['Dateiname']}
            continue
        elif 'Versag' in files[i]:
            matchList[files[i]]={'Versagung':['Dateiname']}
            continue
        elif 'Geneh' in files[i]:
            matchList[files[i]]={'Genehmigung':['Dateiname']}
            continue
        elif 'Zustim' in files[i]:
            matchList[files[i]]={'Genehmigung':['Dateiname']}
            continue
    ##################################################End - Analyse the file name
    ##################################################Begin - look for the required format
    if 'FORMCHECKBOX' in inhalt:
        m+=1
        match={'Antrag':['FORMCHECKBOX']}
        matchList[files[i]]={'Antrag':['FORMCHECKBOX']}
        print('#m: ',m,'Match: ',match)
        continue
    elif 'GeschZ' not in inhalt:
        ############################################
        if pages<=5:
            matchAnfrage=findVorgang(inhalt).anfrage
            if matchAnfrage:#Dokumente die kein GeschZ enthalten, können auch Anfragen sein.
                m+=1
                match={'Anfrage':matchAnfrage}
                matchList[files[i]]={'Anfrage':matchAnfrage}
                print('#m: ',m,'Match: ',match)
            else:
                print('File does not corresponed to the required format')
                wrongFormatList.append(files[i])
        else:
            print('File does not corresponed to the required format')
            wrongFormatList.append(files[i])
        continue
    ##################################################End-look for the required format
    else:
        ########################################################################################################
        #######################################begin remove manipulative content
        searchAuflage = re.search('Auflage[n]*\sund\sBedingung[en]*|Auflage[n]*[^.]*[:]',inhalt)
        if searchAuflage:
            startInhalt=searchAuflage.span()[1]#[1] damit Auflage auch im Match berücksichtigt wird.
            inhalt=inhalt[:startInhalt]
        else:
            searchSachverhalt=re.search('Begründung.*Sachverhalt',inhalt)
            if searchSachverhalt:
                startInhalt=searchSachverhalt.span()[0]
                inhalt=inhalt[:startInhalt]
            else:
                searchAusgleich=re.search('Entscheidung\süber\sden\sAusgleich\sin\sGeld',inhalt)
                if searchAusgleich:
                    startRemove=searchAusgleich.span()[0]
                    inhalt=inhalt[:startRemove]
        #################################################remove "Ausgleich in Geld ist nicht genehmigungsfähig"
        search=re.search('(Ausgleich.+Geld.+gewähren)', inhalt)
        if search:
            startInhalt=search.span()[0]
            stopInhalt=search.span()[1]
            removeInhalt=inhalt[startInhalt:stopInhalt]
            inhalt=inhalt.replace(removeInhalt,'')
        #######################################end remove manipulative content
        ########################################################################################################
        match=findVorgang(inhalt).all()
        if match:
            m+=1
            matchList[files[i]]=match
            print('#m: ',m,'Match: ',match)
            
        else:
            n+=1
            noMatch=findVorgang(inhalt).negativAllList()
            noMatchList[files[i]]=noMatch
            print('#n: ',n,'No-Match: ',noMatch)
            

i:  0 n:  0 m:  0 00_Auslobung_htw-fki_lo.pdf
File does not contain any text data: 00_Auslobung_htw-fki_lo.pdf
i:  1 n:  0 m:  0 02_Zusammenfassung.doc
File does not corresponed to the required format
i:  2 n:  0 m:  0 04_Kunsthistorische_Bedeutung_der_Fernmeldekabelfabrik.PDF
File does not corresponed to the required format
i:  3 n:  0 m:  0 05_Klipp_Oberschöneweide_FHTW8.9.2000.doc
File does not corresponed to the required format
i:  4 n:  0 m:  0 060920_Zustandsbericht-Fassade-A4_sw.pdf
File does not contain any text data: 060920_Zustandsbericht-Fassade-A4_sw.pdf
i:  5 n:  0 m:  0 070927_Vermerk_Abstimmung_Fassade_.pdf
File does not corresponed to the required format
i:  6 n:  0 m:  0 07_LDR Auszug aus der Sitzung 25.11.2005.doc
File does not corresponed to the required format
i:  7 n:  0 m:  0 08_LDR_Kabelfabrik_28.11.05.doc
File does not corresponed to the required format
i:  8 n:  0 m:  0 09_P_SZ_19.12.05.doc
File does not corresponed to the required format
i:  9 n:  0 m:  0 10-0

In [6]:
#Ausschluss-Analyse
#Nicht-Genehmigung means Versagung
for file,val in list(noMatchList.items()):
    #if len(val.keys())==2 and 'Nicht-Genehmigung' in val.keys():
    if 'Nicht-Genehmigung' in val.keys():
        matchList[file]={'Versagung': val['Nicht-Genehmigung']}
        del noMatchList[file]

In [7]:
print('lenMatchList',len(list(matchList.keys())))
print('lenNoMatchList',len(list(noMatchList.keys())))
print('lenWrongFormat',len(wrongFormatList))

lenMatchList 137
lenNoMatchList 30
lenWrongFormat 88


In [8]:
i=0
for el in wrongFormatList:
    print(i,el)
    i+=1

0 00_Auslobung_htw-fki_lo.pdf
1 02_Zusammenfassung.doc
2 04_Kunsthistorische_Bedeutung_der_Fernmeldekabelfabrik.PDF
3 05_Klipp_Oberschöneweide_FHTW8.9.2000.doc
4 060920_Zustandsbericht-Fassade-A4_sw.pdf
5 070927_Vermerk_Abstimmung_Fassade_.pdf
6 07_LDR Auszug aus der Sitzung 25.11.2005.doc
7 08_LDR_Kabelfabrik_28.11.05.doc
8 09_P_SZ_19.12.05.doc
9 10-09-16_HTW_G_Ansicht_2_n.pdf
10 10-09-16_HTW_G_Ansicht_3_n.pdf
11 10-09-16_HTW_G_Ansicht_n.pdf
12 100510-PS_W1_1_Werkplan_Absteckung_BA_1.pdf
13 100510-PS_W1_2_Werkplan_Absteckung_BA_2.pdf
14 10_P_BZ_10.02.06.doc
15 11-09-29_Anwesenheit_Bemusterung_Denkmalamt_bearbeitet.pdf
16 11-09-29_Protokoll_Bemusterung_Denkmalamt_bearbeitet.pdf
17 11_P_TSp_01.03.06.doc
18 120120_Protokoll_12_Abstimmungsrunde_UNESC0_Siedlungen.pdf
19 12_P_FAZ 31.05.06.doc
20 13_Bericht Bauakten Fernmeldekabelfabrik_Demp.doc
21 14_Moldenschardt AdK Kabelwerk Oberspree 09.02.06.doc
22 15_Docomomo12052006.pdf
23 16_ICOMOS.PDF
24 17_BEOS_Junge-Reyer09.05.06.doc
25 18_Wend_D

In [9]:
i=0
filesNoMatch=[]
for k,v in list(noMatchList.items()):
    filesNoMatch.append(str(k))
    print(i,v,k)
    i+=1

0 {'all': []} Abgabenachr.an Ord L_Lärmbelästig..doc
1 {'all': []} Akazienhof 26_Dankschr.f.Fotos.doc
2 {'all': []} Akazienhof, Antrag Sperrung.doc
3 {'all': []} Akazienhof, Leuhcten Info LDA.doc
4 {'all': []} Anschreibenkorrektur.doc
5 {'all': []} Fläche Liegenschaftsfondr.doc
6 {'all': []} Konzept Silicon Sensor FHTW_an Dr.Dunger.doc
7 {'all': []} Kopie von WHLBLEG.DOC
8 {'all': []} Maststandorte.doc
9 {'all': []} Mitteilg.BzStr.Schneider_Nichtteilnahme an Einweihung.doc
10 {'all': []} Schr.an Fr.Jeske,FTHW_Adresse v.UD.doc
11 {'all': []} Schr.an Hr.Walter_Lärmbelästigung.doc
12 {'all': []} Schr.an LDA_Bestätig.Dokumentation.doc
13 {'all': []} Vermerk zu A2-Gebäude_Brüstungsgeländer.doc
14 {'all': []} W'hof 76,77(Rathn.-Villa)Antr.auf Erteilg.eines Negativz..doc
15 {'all': []} W'hof 76-77 - Abstimmg.Fensterprofile.doc
16 {'all': []} W'hofstr.75A_Nachtr.unterl.z.Demonstrat.geb..doc
17 {'all': []} W'hofstr.76-77_Abriss Halle 19.doc
18 {'all': []} WHL76_77.DOC
19 {'all': []} WHLBLEG.DOC

In [10]:
#Hierarchie-Analyse
#solves uncertain classification
for file,val in  list(matchList.items()):
    if len(val.keys())>1:
        ###Anfrage=question, Nachforderung=claim can contain information about Versagung or Genehmgigung. However, it is still a question or a claim.
        if ('Anfrage' or 'Nachforderung') in val.keys() and len(val.keys())>1:
            for key in list(val.keys()):
                if key!='Anfrage' and key!='Nachforderung':
                    del matchList[file][key]
        if 'Eingang' in val.keys() and len(val.keys())>1:
            for key in list(val.keys()):
                if key!='Eingang':
                    del matchList[file][key]
        ###Anhörung includes also Versagung but not reverse
        if len(val.keys())==2 and ('Anhörung' and 'Versagung') in val.keys():
            del matchList[file]['Versagung']
        if len(val.keys())==2 and ('Genehmigung' and 'Versagung') in val.keys():
            del matchList[file]['Genehmigung']
        if len(val.keys())==2 and ('Genehmigung' and 'Anhörung') in val.keys():
            del matchList[file]['Genehmigung']
        if len(val.keys())==3 and ('Genehmigung' and 'Versagung' and 'Anhörung') in val.keys():
            del matchList[file]['Genehmigung']
            del matchList[file]['Versagung']
        ###Bauverfahren is relatively uncertain. It appiers mostly with Genehmigung.Genehmigung is more certain
        if 'Bauverfahren' in val.keys():
            del matchList[file]['Bauverfahren']

In [11]:
#summarize the results
i=0
points={'Gesamt-Zuordnung':0,'Informelles-Format':len(wrongFormatList),'Keine-Zuordnung':len(noMatchList.keys()),'Genehmigung':0,'Versagung':0,'Anhörung':0,'Antrag':0,'Anfrage':0,'Nachforderung':0,'Bauverfahren':0,'Eingang':0}
for k,v in list(matchList.items()):
    if len(v.keys())>0:
        print(i,v,k)
        i+=1
    points['Gesamt-Zuordnung']+=1  
    for key in points.keys():
        if key in v.keys():
            points[key]+=1
print(points)

0 {'Antrag': ['FORMCHECKBOX']} 152-13,W'hofstr.74-76_Errichtung Werbung.doc
1 {'Antrag': ['FORMCHECKBOX']} 158-14,W'hofstr.75_Inselnetztestanlage.doc
2 {'Anfrage': ['Dateiname']} 1mündliche BVV-Anfrage_Abriss A8-Geb. Ziesel-Bau.doc
3 {'Antrag': ['FORMCHECKBOX']} 239-14,W'hofstr.78_Umspannwerk-Dachreparatur.doc
4 {'Antrag': ['FORMCHECKBOX']} 241-13,W'hofstr.75A_Temporäre Errichtg.Kleinstwindanl..doc
5 {'Antrag': ['FORMCHECKBOX']} 291-12,W'hofstr.76-78_Neubau Kultur-Abstimmg.Farbe.doc
6 {'Antrag': ['FORMCHECKBOX']} 299-13,Akazienhof 15_Abriss Schuppenanbau.doc
7 {'Antrag': ['FORMCHECKBOX']} 363-12,W'hofstr.75a_Einbau Schlüsseldep.f.Feuerwehr.doc
8 {'Antrag': ['FORMCHECKBOX']} 385-12,W'hofstr.76,77_Erneuerg.Schmutzwasserkanal.doc
9 {'Antrag': ['FORMCHECKBOX']} 50-14,W'hofstr.78_Nutzg.änderg.Umspannwerk z.Bronzegießerei.doc
10 {'Antrag': ['FORMCHECKBOX']} 685-12,W'hofstr.75A_Einbau Solaranlage.doc
11 {'Antrag': ['FORMCHECKBOX']} 733-12,Akazienhof 15_IS Doppelschuppen.doc
12 {'Antrag': ['FO

In [12]:
#summarize the results
resultDict={}
resultDict['Ergebnis']=points
resultDict['Gesamt-Zuordnung']=matchList
resultDict['Keine-Zuordnung']=noMatchList
resultDict['Informelles-Format']=wrongFormatList

In [13]:
#save objNrValue as json file
path=r'C:\Users\guendm1\Documents\BerlinDenkmal\Denkmalliste\Vorgang\\'
#with open(path+'resultVorgangAll-NoDocName.json', 'w') as fp:
with open(path+'resultVorgangAll.json', 'w') as fp:
    json.dump(resultDict, fp,  indent=4,ensure_ascii=False)
print('successfully saved')

successfully saved
